<a href="https://colab.research.google.com/github/NakbarDGOAT/UFV-visualizacion/blob/main/Ejercicios_de_clase/PracticaPython04112024_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import plotly.express as px
from google.colab import files

In [2]:
# Subir archivo de datos
uploaded = files.upload()

Saving datos_ejercicio_ventas.csv to datos_ejercicio_ventas.csv


In [3]:
# Cargar el archivo
data = pd.read_csv(list(uploaded.keys())[0])

# Parte 0: Familiarizarnos con la base de datos

Esta parte no la he hecho yo, se la he pedido a un compañero ya que no estuve en clase el día que lo hicisteis y no tenía otra manera de tenerlo

In [4]:
# Revisar el tipo de datos de las columnas
print(data.dtypes)

# Asegurarse de que los tipos sean consistentes
# Convertir la columna 'FORECAST_YEAR' a cadena si es necesario
data['FORECAST_YEAR'] = data['FORECAST_YEAR'].astype(str)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT           float64
dtype: object


In [5]:
# Contar los valores en la columna 'COUNTRY'
paises_counts = data['COUNTRY'].value_counts()

# Mostrar el resultado
print(paises_counts)

COUNTRY
Great Britain    4008
Netherlands      2476
Hungary          2384
Denmark          2294
Spain            2152
Norway           2062
Czech            1403
Portugal         1179
Italy             708
Name: count, dtype: int64


In [6]:
# Contar los valores en la columna 'COUNTRY'
productos_counts = data['SUBBRAND'].value_counts()

# Mostrar el resultado
print(productos_counts)

SUBBRAND
Pepsi Max (L3)        5598
Pepsi Regular (L3)    3636
7up (L3)              2694
Lipton (L3)           2693
7up Free (L3)         2343
Mountain Dew (L3)     1702
Name: count, dtype: int64


In [7]:
portugal_lipton = data[(data['COUNTRY'] == 'Portugal') & (data['SUBBRAND'] == 'Lipton (L3)')]

# Mostrar las primeras filas del nuevo DataFrame
print(portugal_lipton.head())

      COUNTRY     SUBBRAND  YEAR  MONTH     SCENARIO FORECAST FORECAST_YEAR  \
0    Portugal  Lipton (L3)  2023     12  AI_forecast  AI_P02F        2023.0   
292  Portugal  Lipton (L3)  2023     10       actual      NaN           nan   
454  Portugal  Lipton (L3)  2024      9  AI_forecast  AI_P04F        2023.0   
490  Portugal  Lipton (L3)  2023      7  AI_forecast  AI_P04F        2023.0   
530  Portugal  Lipton (L3)  2023      9  AI_forecast  AI_P06F        2023.0   

           AMOUNT  
0    7.543562e+05  
292  1.206886e+06  
454  7.122028e+05  
490  8.897866e+05  
530  7.676745e+05  


In [8]:
# Filtrar para obtener solo los escenarios actuales y crear una copia para evitar el SettingWithCopyWarning
historico_forecast = portugal_lipton[portugal_lipton['SCENARIO'] == 'actual'].copy()

# Combinar YEAR y MONTH en una nueva columna de fecha
historico_forecast.loc[:, 'fecha_actual'] = pd.to_datetime(
    historico_forecast['YEAR'].astype(str) + '-' + historico_forecast['MONTH'].astype(str) + '-01'
)

# Encontrar la primera fecha solo con mes y año
primera_fecha_actual = historico_forecast['fecha_actual'].min().strftime('%Y-%m')

# Encontrar la última fecha solo con mes y año
ultima_fecha_actual = historico_forecast['fecha_actual'].max().strftime('%Y-%m')

# Mostrar los resultados
print("Primera fecha actual (Año-Mes):", primera_fecha_actual)
print("Última fecha actual (Año-Mes):", ultima_fecha_actual)


Primera fecha actual (Año-Mes): 2023-01
Última fecha actual (Año-Mes): 2024-08


In [9]:
# Filtrar para obtener solo los escenarios actuales y crear una copia para evitar el SettingWithCopyWarning
historico_forecast_prim = portugal_lipton[portugal_lipton['FORECAST'] == 'AI_P02F'].copy()

# Combinar YEAR y MONTH en una nueva columna de fecha
historico_forecast_prim.loc[:, 'fecha_actual'] = pd.to_datetime(historico_forecast_prim['YEAR'].astype(str) + '-' + historico_forecast_prim['MONTH'].astype(str) + '-01')

# Encontrar la primera fecha solo con mes y año
primera_fecha_prim = historico_forecast_prim['fecha_actual'].min().strftime('%Y-%m')

# Filtrar para obtener solo los escenarios actuales y crear una copia para evitar el SettingWithCopyWarning
historico_forecast_ult = portugal_lipton[portugal_lipton['FORECAST'] == 'AI_PF'].copy()

# Combinar YEAR y MONTH en una nueva columna de fecha
historico_forecast_ult.loc[:, 'fecha_actual'] = pd.to_datetime(historico_forecast_ult['YEAR'].astype(str) + '-' + historico_forecast_ult['MONTH'].astype(str) + '-01')

# Encontrar la última fecha solo con mes y año
ultima_fecha_ultima = historico_forecast_ult['fecha_actual'].max().strftime('%Y-%m')

# Mostrar los resultados
print("Primera fecha forecast (Año-Mes):", primera_fecha_prim)
print("Última fecha forecast (Año-Mes):", ultima_fecha_ultima)


Primera fecha forecast (Año-Mes): 2023-02
Última fecha forecast (Año-Mes): 2024-06


# Pregunta 1: Distribución de ventas

In [10]:
# Ventas por país
sales_by_country = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index() #Sumamos las ventas por país (Utilizamos reset index para convertir el indice a columna)
fig_country = px.bar(sales_by_country, x='COUNTRY', y='AMOUNT', title='Distribución de Ventas por País')
fig_country.show()

Representamos los datos tambien en formato PieChart, así es más clara la comparación entre los diferentes paises

In [11]:
# Ordenamos los datos para poder separar los top 4 paises con más ventas
sales_by_country = sales_by_country.sort_values(by='AMOUNT', ascending=False)

# Agrupamos en others el resto de países
top_5 = sales_by_country[:4]
others = pd.DataFrame([{
    'COUNTRY': 'Others',
    'AMOUNT': sales_by_country[4:]['AMOUNT'].sum()}])

# Los concatenamos
sales_by_country_top4 = pd.concat([top_5, others])

fig_country = px.pie(sales_by_country_top4,
                     names='COUNTRY',
                     values='AMOUNT',
                     title='Distribución de Ventas por País (Top 4 y Otros)')
fig_country.show()


In [12]:
# Ventas por fecha (Año y Mes combinados)
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1)) #Creamos la fecha completa, asignamos day=1 para que tome cada mes como el dia 1
sales_by_date = data.groupby('DATE')['AMOUNT'].sum().reset_index() #Calculamos el total de ventas por mes
fig_date = px.line(sales_by_date, x='DATE', y='AMOUNT', title='Distribución de Ventas por Mes y Año')
fig_date.show()

In [13]:
# Ventas por marca
sales_by_brand = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index() #Calculamos el total de ventas por marca
fig_brand = px.bar(sales_by_brand, x='SUBBRAND', y='AMOUNT', title='Distribución de Ventas por Marca')
fig_brand.show()

Al igual que hemos hecho en el apartado anterior, crearemos una Pie Chart para ver realmente la comparación entre las marcas.

In [14]:
sales_by_brand = sales_by_brand.sort_values(by='AMOUNT', ascending=False)

top_4 = sales_by_brand[:4]
others = pd.DataFrame([{
    'SUBBRAND': 'Others',
    'AMOUNT': sales_by_brand[4:]['AMOUNT'].sum()
}])

sales_by_brand_top4 = pd.concat([top_4, others])

fig_brand = px.pie(sales_by_brand_top4,
                   names='SUBBRAND',
                   values='AMOUNT',
                   title='Distribución de Ventas por Marca (Top 4 y Otros)')

fig_brand.show()

# Pregunta 2: Tendencia y estacionalidad

Para analizar la tendencia se ha elegido calcular la media móvil de los últimos 5 meses, con esto podremos ver como va variando la tendencia, al elegir los últimos 5 meses la tendencia se ajustará más a los últimos datos y nos permitirá evaluarlo mejor.

In [26]:
# Convertir columna de fecha para trabajar con fechas completas
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# Encontrar la marca con más ventas
sales_by_brand = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
brand_most_sales = sales_by_brand[sales_by_brand['AMOUNT'] == sales_by_brand['AMOUNT'].max()]['SUBBRAND'].values[0]

# Filtrar datos para la marca con más ventas
data_most_brand = data[data['SUBBRAND'] == brand_most_sales]
sales_most_brand_date = data_most_brand.groupby('DATE')['AMOUNT'].sum().reset_index()

# Calcular la tendencia usando un promedio móvil de 5 meses
sales_most_brand_date['Trend'] = sales_most_brand_date['AMOUNT'].rolling(window=5, center=True).mean()

# Calcular la estacionalidad como la diferencia entre el valor original y la tendencia
sales_most_brand_date['Seasonal'] = sales_most_brand_date['AMOUNT'] - sales_most_brand_date['Trend']

# Filtrar los datos hasta agosto de 2024
sales_most_brand_date = sales_most_brand_date[sales_most_brand_date['DATE'] <= '2024-09-01']

# Usar plotly.express para crear un gráfico de líneas
fig = px.line(sales_most_brand_date, x='DATE', y=['AMOUNT', 'Trend'],
              labels={'value': 'Monto de Ventas', 'variable': 'Serie'},
              title=f'Tendencia y Estacionalidad de la Marca {brand_most_sales}')

# Mostrar el gráfico
fig.show()


No tendremos en cuenta los datos a partir de septiemdre de 2024 porque dejan de ser datos actuales y no representan lo mismo que los datos anteriores


Conclusiones: El gráfico muestra una tendencia ascendente, con un claro crecimiento desde enero de 2023 y alcanzando su máximo a mediados de 2024.

Ahora haremos un mapa de calor con el que veremos la estacionalidad.

In [16]:
# Filtrar datos para el país con menos ventas y crear una copia para evitar el SettingWithCopyWarning
data_least_country = data[data['COUNTRY'] == country_least_sales].copy()

# Agregar columnas de Año y Mes para el análisis estacional
data_least_country.loc[:, 'Year'] = data_least_country['DATE'].dt.year
data_least_country.loc[:, 'Month'] = data_least_country['DATE'].dt.month

# Eliminar los datos del año 2025
data_least_country = data_least_country[data_least_country['Year'] != 2025]

# Crear una columna que agrupe los meses en cuatrimestres: 1-4, 5-8, y 9-12
def month_group(month):
    if 1 <= month <= 4:
        return '1-4'
    elif 5 <= month <= 8:
        return '5-8'
    else:
        return '9-12'

data_least_country.loc[:, 'Month_Group'] = data_least_country['Month'].apply(month_group)

# Calcular las ventas totales para cada combinación de Año y Grupo de Meses
monthly_sales_grouped = data_least_country.groupby(['Year', 'Month_Group'])['AMOUNT'].sum().reset_index()

# Crear el mapa de calor
fig = px.density_heatmap(monthly_sales_grouped, x='Month_Group', y='Year', z='AMOUNT',
                         color_continuous_scale='Viridis',
                         labels={'AMOUNT': 'Monto de Ventas', 'Month_Group': 'Grupo de Meses', 'Year': 'Año'},
                         title=f'Mapa de Calor de Ventas por Grupo de Meses en {country_least_sales}')

# Mostrar el gráfico
fig.show()


Podemos ver como existe una estacionalidad bi-anual, con picos de ventas en los meses de verano y en los meses de navidad. Queda evidente a la vista que el pico de navidad es superior al pico que se obtiene en los meses de verano.
Los datos que tenemos no nos permiten confirmar que en los meses de navidad existe otro pico en el año 2024, pero es de suponer que esto sucederá.

In [25]:
# Convertir columna de fecha para trabajar con fechas completas
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# Encontrar la marca con más ventas
sales_by_brand = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
brand_most_sales = sales_by_brand[sales_by_brand['AMOUNT'] == sales_by_brand['AMOUNT'].max()]['SUBBRAND'].values[0]

# Filtrar datos para la marca con más ventas
data_most_brand = data[data['SUBBRAND'] == brand_most_sales]
sales_most_brand_date = data_most_brand.groupby('DATE')['AMOUNT'].sum().reset_index()

# Calcular la tendencia usando un promedio móvil de 5 meses
sales_most_brand_date['Trend'] = sales_most_brand_date['AMOUNT'].rolling(window=5, center=True).mean()

# Filtrar los datos hasta agosto de 2024
sales_most_brand_date = sales_most_brand_date[sales_most_brand_date['DATE'] <= '2024-09-01']

# Usar plotly.express para crear un gráfico de líneas
fig = px.line(sales_most_brand_date, x='DATE', y=['AMOUNT', 'Trend'],
              labels={'value': 'Monto de Ventas', 'variable': 'Serie'},
              title=f'Tendencia y Estacionalidad de la Marca {brand_most_sales}')

# Mostrar el gráfico
fig.show()


No tendremos en cuenta los datos a partir de septiemdre de 2024 dejan de ser datos actuales y no representan lo mismo que los datos anteriores

Conclusiones: El gráfico de la marca Pepsi es relamente parecido al de las ventas en españa, lo cuál nos hace sacar las mismas conclusiones que en el apartado anterior.


In [18]:
# Filtrar datos para la marca con más ventas
data_most_brand = data[data['SUBBRAND'] == brand_most_sales].copy()  # Usamos .copy() para evitar el SettingWithCopyWarning

# Agregar columnas de Año y Mes para el análisis estacional
data_most_brand.loc[:, 'Year'] = data_most_brand['DATE'].dt.year
data_most_brand.loc[:, 'Month'] = data_most_brand['DATE'].dt.month

# Eliminar los datos del año 2025
data_most_brand = data_most_brand[data_most_brand['Year'] != 2025]

# Crear una columna que agrupe los meses en cuatrimestres: 1-4, 5-8, y 9-12
def month_group(month):
    if 1 <= month <= 4:
        return '1-4'
    elif 5 <= month <= 8:
        return '5-8'
    else:
        return '9-12'

data_most_brand.loc[:, 'Month_Group'] = data_most_brand['Month'].apply(month_group)

# Calcular las ventas totales para cada combinación de Año y Grupo de Meses
monthly_sales_grouped = data_most_brand.groupby(['Year', 'Month_Group'])['AMOUNT'].sum().reset_index()

# Crear el mapa de calor
fig = px.density_heatmap(monthly_sales_grouped, x='Month_Group', y='Year', z='AMOUNT',
                         color_continuous_scale='Viridis',
                         labels={'AMOUNT': 'Monto de Ventas', 'Month_Group': 'Grupo de Meses', 'Year': 'Año'},
                         title=f'Mapa de Calor de Ventas por Grupo de Meses de la Marca {brand_most_sales}')

# Mostrar el gráfico
fig.show()

Las conclusiones son las mismas que las obtenidas anteriormente, existe una bi-estacionalidad con picos en los meses de verano y en los de navidad. Se puede observar que en la amrca pespi también el pico es más alto en los meses de navidad.
Al igual que en el apartado anterior, no podemos confirmar que en navidad de 2024 habrá un pico superior al de verano de 2024, pero si todo sigue según lo previsto, esto es lo que sucederá.

# Pregunta 3: Predicciones en España

La base de datos tiene duplicados en todas las predicciones excepto en P02F y P03F para mostrarlo primero se enseñarán las diferencias entre una de esas predicciones y una que si contenga duplicados. Luego quitaremos los duplicados de la predicción P07F para ver la diferencia. Finalmente, intentaremos calcular la mejor predicción posible.

In [19]:
# Filtrar datos reales desde enero de 2023 hasta agosto de 2024 para el país Spain y escenario actual
ventas_reales = data[(data['YEAR'] == 2023) & (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain') |
                     ((data['YEAR'] == 2024) & (data['MONTH'] <= 8) & (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain'))]

ventas_reales_sum = ventas_reales.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Filtrar predicciones AI_P07F en los años 2023 y 2024 hasta agosto
prediccion_2023_2024 = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                            (data['FORECAST'] == 'AI_P07F') & (data['COUNTRY'] == 'Spain')]

prediccion_2023_2024_sum = prediccion_2023_2024.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Unir los dos DataFrames para conservar solo los meses que tienen datos en ambas series
merged_data = pd.merge(ventas_reales_sum, prediccion_2023_2024_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_forecast'))

# Crear un gráfico de líneas con las dos series separadas
fig = px.line(title='Comparación de Ventas Reales y Predicción AI_P07F')

# Añadir la línea de ventas reales
fig.add_scatter(x=merged_data['MONTH'] + (merged_data['YEAR'] - 2023) * 12,
                y=merged_data['AMOUNT_real'], mode='lines', name='Ventas Reales 2023-2024')

# Añadir la línea de la predicción
fig.add_scatter(x=merged_data['MONTH'] + (merged_data['YEAR'] - 2023) * 12,
                y=merged_data['AMOUNT_forecast'], mode='lines', name='Predicción AI_P07F 2023-2024')

# Configurar etiquetas de los ejes
fig.update_layout(xaxis_title='Mes', yaxis_title='Monto de Ventas',
                  xaxis=dict(tickvals=[i for i in range(1, 21)],
                             ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                       'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

In [20]:
# Filtrar datos reales desde enero de 2023 hasta agosto de 2024 para el país Spain y escenario actual
ventas_reales = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')]
ventas_reales_sum = ventas_reales.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Filtrar predicciones AI_P02F en los años 2023 y 2024 hasta agosto
prediccion_2023_2024 = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                            (data['FORECAST'] == 'AI_P02F') & (data['COUNTRY'] == 'Spain')]
prediccion_2023_2024_sum = prediccion_2023_2024.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Unir los dos DataFrames para conservar solo los meses que tienen datos en ambas series
merged_data = pd.merge(ventas_reales_sum, prediccion_2023_2024_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_forecast'))

# Crear un gráfico de líneas con las dos series separadas
fig = px.line(title='Comparación de Ventas Reales y Predicción AI_P02F')

# Añadir la línea de ventas reales
fig.add_scatter(x=merged_data['MONTH'] + (merged_data['YEAR'] - 2023) * 12,
                y=merged_data['AMOUNT_real'], mode='lines', name='Ventas Reales 2023-2024')

# Añadir la línea de la predicción
fig.add_scatter(x=merged_data['MONTH'] + (merged_data['YEAR'] - 2023) * 12,
                y=merged_data['AMOUNT_forecast'], mode='lines', name='Predicción AI_P02F 2023-2024')

# Configurar etiquetas de los ejes
fig.update_layout(xaxis_title='Mes', yaxis_title='Monto de Ventas',
                  xaxis=dict(tickvals=[i for i in range(1, 21)],
                             ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                       'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

# Mostrar el gráfico
fig.show()

A simple vista se ve que la predicción AI_P02F es mucho mejor, pero igualmente vamos a calcular el error absoluto de ambas predicciones y vamos a comparar los resultados en los meses en los que existan ambas predicciones

In [27]:
# Filtrar datos reales desde enero de 2023 hasta agosto de 2024 para el país Spain y escenario actual
ventas_reales = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')]
ventas_reales_sum = ventas_reales.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Filtrar predicciones AI_P07F en los años 2023 y 2024 hasta agosto
prediccion_AI_P07F = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                          (data['FORECAST'] == 'AI_P07F') & (data['COUNTRY'] == 'Spain')]
prediccion_AI_P07F_sum = prediccion_AI_P07F.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Filtrar predicciones AI_P02F en los años 2023 y 2024 hasta agosto
prediccion_AI_P02F = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                          (data['FORECAST'] == 'AI_P02F') & (data['COUNTRY'] == 'Spain')]
prediccion_AI_P02F_sum = prediccion_AI_P02F.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Unir datos reales con cada predicción para calcular los errores
merged_data_P07F = pd.merge(ventas_reales_sum, prediccion_AI_P07F_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_P07F'))
merged_data_P02F = pd.merge(ventas_reales_sum, prediccion_AI_P02F_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_P02F'))

# Calcular el error absoluto para cada predicción
merged_data_P07F['Error_Absoluto_P07F'] = abs(merged_data_P07F['AMOUNT_real'] - merged_data_P07F['AMOUNT_P07F'])
merged_data_P02F['Error_Absoluto_P02F'] = abs(merged_data_P02F['AMOUNT_real'] - merged_data_P02F['AMOUNT_P02F'])

# Unir ambos errores en un solo DataFrame para el gráfico de barras
error_comparison = pd.merge(merged_data_P07F[['YEAR', 'MONTH', 'Error_Absoluto_P07F']],
                            merged_data_P02F[['YEAR', 'MONTH', 'Error_Absoluto_P02F']],
                            on=['YEAR', 'MONTH'])

# Preparar el formato para un gráfico de barras apilado
error_comparison = error_comparison.melt(id_vars=['YEAR', 'MONTH'],
                                         value_vars=['Error_Absoluto_P07F', 'Error_Absoluto_P02F'],
                                         var_name='Tipo_Prediccion', value_name='Error_Absoluto')

# Crear un gráfico de barras comparativo de los errores absolutos
fig_error = px.bar(error_comparison,
                   x=error_comparison['MONTH'] + (error_comparison['YEAR'] - 2023) * 12,
                   y='Error_Absoluto',
                   color='Tipo_Prediccion',
                   title='Comparación de Error Absoluto Mensual entre Predicciones AI_P07F y AI_P02F para 2023-2024',
                   labels={'x': 'Mes', 'Error_Absoluto': 'Error Absoluto'},
                   barmode='group',  # Agrupar barras
                   template='plotly_dark')

# Configurar etiquetas de los ejes
fig_error.update_layout(xaxis_title='Mes', yaxis_title='Error Absoluto',
                        xaxis=dict(tickvals=[i for i in range(1, 21)],
                                   ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                       'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

# Mostrar el gráfico
fig_error.show()

Sin duda las conclusiones obtenidas son las esperadas, la predicción AI_P02F es muchísimo mejor que la predicción AI_P07F, debido a que esta contiene duplicados.

Ahora eliminaremos los duplicados y compararemos los errores absolutos de P07F con duplicados y sin ellos

In [32]:
# Filtrar datos reales desde enero de 2023 hasta agosto de 2024 para el país Spain y escenario actual
ventas_reales = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')]
ventas_reales_sum = ventas_reales.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Filtrar predicciones AI_P07F en los años 2023 y 2024 hasta agosto
prediccion_2023_2024 = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                            (data['FORECAST'] == 'AI_P07F') & (data['COUNTRY'] == 'Spain')]

# Verificar si hay duplicados en la predicción
duplicates = prediccion_2023_2024.duplicated(subset=['YEAR', 'MONTH', 'SUBBRAND'], keep=False)

if duplicates.any():
    # Función para seleccionar el valor más cercano a la mediana en cada grupo duplicado
    def select_nearest_to_median(df):
        median_val = df['AMOUNT'].median()  # Calcular la mediana del grupo
        # Seleccionar la fila cuyo valor de AMOUNT esté más cercano a la mediana
        return df.loc[(df['AMOUNT'] - median_val).abs().idxmin()]

    # Aplicar la función para eliminar duplicados y mantener solo la fila más cercana a la mediana
    prediccion_2023_2024_unique = prediccion_2023_2024.groupby(['YEAR', 'MONTH', 'SUBBRAND'], group_keys=False).apply(select_nearest_to_median).reset_index(drop=True)
else:
    # Si no hay duplicados, usar los datos tal como están
    prediccion_2023_2024_unique = prediccion_2023_2024.copy()

# Agrupar las predicciones para obtener la suma por mes y año
prediccion_2023_2024_sum = prediccion_2023_2024_unique.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Unir los datos reales y de predicción para los meses con datos en ambas series
merged_data = pd.merge(ventas_reales_sum, prediccion_2023_2024_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_forecast'))

# Crear un gráfico de líneas con las dos series separadas
fig = px.line(title='Comparación de Ventas Reales y Predicción AI_P07F (sin duplicados)')

# Añadir la línea de ventas reales
fig.add_scatter(x=merged_data['MONTH'] + (merged_data['YEAR'] - 2023) * 12,
                y=merged_data['AMOUNT_real'], mode='lines', name='Ventas Reales 2023-2024')

# Añadir la línea de la predicción
fig.add_scatter(x=merged_data['MONTH'] + (merged_data['YEAR'] - 2023) * 12,
                y=merged_data['AMOUNT_forecast'], mode='lines', name='Predicción AI_P07F 2023-2024')

# Configurar etiquetas de los ejes
fig.update_layout(xaxis_title='Mes', yaxis_title='Monto de Ventas',
                  xaxis=dict(tickvals=[i for i in range(1, 21)],
                             ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                       'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

# Mostrar el gráfico
fig.show()


<ipython-input-32-5bad77b52077>:21: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [30]:
# Filtrar datos reales desde enero de 2023 hasta agosto de 2024 para el país Spain y escenario actual
ventas_reales = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')]
ventas_reales_sum = ventas_reales.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Filtrar predicciones AI_P07F con duplicados en los años 2023 y 2024 hasta agosto
prediccion_AI_P07F_con_duplicados = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                                         (data['FORECAST'] == 'AI_P07F') & (data['COUNTRY'] == 'Spain')]
prediccion_AI_P07F_con_duplicados_sum = prediccion_AI_P07F_con_duplicados.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Calcular el error absoluto con duplicados
merged_data_P07F_con_duplicados = pd.merge(ventas_reales_sum, prediccion_AI_P07F_con_duplicados_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_P07F'))
merged_data_P07F_con_duplicados['Error_Absoluto_P07F_con_duplicados'] = abs(merged_data_P07F_con_duplicados['AMOUNT_real'] - merged_data_P07F_con_duplicados['AMOUNT_P07F'])

# Eliminar duplicados en AI_P07F seleccionando el valor más cercano a la mediana
def select_nearest_to_median(df):
    median_val = df['AMOUNT'].median()
    return df.loc[(df['AMOUNT'] - median_val).abs().idxmin()]

if prediccion_AI_P07F_con_duplicados.duplicated(subset=['YEAR', 'MONTH', 'SUBBRAND'], keep=False).any():
    prediccion_AI_P07F_sin_duplicados = prediccion_AI_P07F_con_duplicados.groupby(['YEAR', 'MONTH', 'SUBBRAND']).apply(select_nearest_to_median).reset_index(drop=True)
else:
    prediccion_AI_P07F_sin_duplicados = prediccion_AI_P07F_con_duplicados.copy()

# Agrupar las predicciones sin duplicados para obtener la suma por mes y año
prediccion_AI_P07F_sin_duplicados_sum = prediccion_AI_P07F_sin_duplicados.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Calcular el error absoluto sin duplicados
merged_data_P07F_sin_duplicados = pd.merge(ventas_reales_sum, prediccion_AI_P07F_sin_duplicados_sum, on=['YEAR', 'MONTH'], suffixes=('_real', '_P07F'))
merged_data_P07F_sin_duplicados['Error_Absoluto_P07F_sin_duplicados'] = abs(merged_data_P07F_sin_duplicados['AMOUNT_real'] - merged_data_P07F_sin_duplicados['AMOUNT_P07F'])

# Unir ambos errores en un solo DataFrame para el gráfico de barras
error_comparison = pd.merge(
    merged_data_P07F_con_duplicados[['YEAR', 'MONTH', 'Error_Absoluto_P07F_con_duplicados']],
    merged_data_P07F_sin_duplicados[['YEAR', 'MONTH', 'Error_Absoluto_P07F_sin_duplicados']],
    on=['YEAR', 'MONTH']
)

# Preparar el formato para un gráfico de barras agrupado
error_comparison = error_comparison.melt(id_vars=['YEAR', 'MONTH'],
                                         value_vars=['Error_Absoluto_P07F_con_duplicados', 'Error_Absoluto_P07F_sin_duplicados'],
                                         var_name='Tipo_Prediccion', value_name='Error_Absoluto')

# Crear un gráfico de barras comparativo de los errores absolutos
fig_error = px.bar(error_comparison,
                   x=error_comparison['MONTH'] + (error_comparison['YEAR'] - 2023) * 12,
                   y='Error_Absoluto',
                   color='Tipo_Prediccion',
                   title='Comparación de Error Absoluto Mensual entre Predicción AI_P07F con y sin duplicados',
                   labels={'x': 'Mes', 'Error_Absoluto': 'Error Absoluto'},
                   barmode='group',  # Agrupar barras
                   template='plotly_dark')

# Configurar etiquetas de los ejes
fig_error.update_layout(xaxis_title='Mes', yaxis_title='Error Absoluto',
                        xaxis=dict(tickvals=[i for i in range(1, 21)],
                                   ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                             'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

# Mostrar el gráfico
fig_error.show()


<ipython-input-30-46f720e5487c>:21: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Podemos ver como el error de los duplicados es enorme y eliminándolos obtenemos una predicción muhcísimo mejor

In [40]:
# Filtrar datos reales desde enero de 2023 hasta agosto de 2024 para el país Spain y escenario actual
ventas_reales = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')]
ventas_reales_sum = ventas_reales.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Lista de predicciones
predicciones = ['AI_PF'] + [f'AI_P{str(i).zfill(2)}F' for i in range(2, 13)]

# Función para eliminar duplicados seleccionando el valor más cercano a la mediana
def eliminar_duplicados_con_mediana(df):
    def select_nearest_to_median(group):
        median_val = group['AMOUNT'].median()
        return group.loc[(group['AMOUNT'] - median_val).abs().idxmin()]

    # Verificar si hay duplicados y aplicar el filtro si es necesario
    duplicates = df.duplicated(subset=['YEAR', 'MONTH', 'SUBBRAND'], keep=False)
    if duplicates.any():
        # Aplicar el filtro con group_keys=False y resetear el índice
        return df.groupby(['YEAR', 'MONTH', 'SUBBRAND'], group_keys=False).apply(select_nearest_to_median).reset_index(drop=True)
    else:
        return df

# Crear una lista para almacenar DataFrames con el error absoluto de cada predicción
errores_por_prediccion = []

# Calcular el error absoluto para cada predicción después de eliminar duplicados
for pred in predicciones:
    # Filtrar los datos de la predicción específica
    pred_data = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['FORECAST'] == pred) & (data['COUNTRY'] == 'Spain')]

    # Eliminar duplicados de la predicción usando la función definida
    pred_data_sin_duplicados = eliminar_duplicados_con_mediana(pred_data)

    # Agrupar la predicción por mes y año
    pred_data_sum = pred_data_sin_duplicados.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

    # Calcular el error absoluto entre la predicción y los datos reales
    merged_data = pd.merge(ventas_reales_sum, pred_data_sum, on=['YEAR', 'MONTH'], suffixes=('_real', f'_{pred}'))
    merged_data[f'Error_Absoluto_{pred}'] = abs(merged_data['AMOUNT_real'] - merged_data[f'AMOUNT_{pred}'])

    # Guardar el DataFrame de errores para esta predicción
    errores_por_prediccion.append(merged_data[['YEAR', 'MONTH', f'Error_Absoluto_{pred}']])

# Combinar todos los errores en un solo DataFrame
errores_df = errores_por_prediccion[0]
for df in errores_por_prediccion[1:]:
    errores_df = pd.merge(errores_df, df, on=['YEAR', 'MONTH'], how='outer')

# Crear la "mezcla" de predicciones seleccionando la de menor error absoluto para cada mes
errores_df['Error_Absoluto_Mezcla'] = errores_df[[f'Error_Absoluto_{pred}' for pred in predicciones]].min(axis=1)

# Identificar la predicción con el menor error para cada mes
errores_df['Prediccion_Optima'] = errores_df[[f'Error_Absoluto_{pred}' for pred in predicciones]].idxmin(axis=1)
errores_df['Prediccion_Optima'] = errores_df['Prediccion_Optima'].str.replace('Error_Absoluto_', '')

# Preparar el DataFrame para el gráfico de barras
error_comparison = errores_df.melt(id_vars=['YEAR', 'MONTH'],
                                   value_vars=[f'Error_Absoluto_{pred}' for pred in predicciones] + ['Error_Absoluto_Mezcla'],
                                   var_name='Tipo_Prediccion', value_name='Error_Absoluto')

# Cambiar el nombre de la columna 'Tipo_Prediccion' para que se vea más claro en el gráfico
error_comparison['Tipo_Prediccion'] = error_comparison['Tipo_Prediccion'].str.replace('Error_Absoluto_', '')

# Crear un gráfico de barras comparativo de los errores absolutos
fig_error = px.bar(error_comparison,
                   x=error_comparison['MONTH'] + (error_comparison['YEAR'] - 2023) * 12,
                   y='Error_Absoluto',
                   color='Tipo_Prediccion',
                   title='Comparación de Error Absoluto Mensual entre Predicciones y la Mezcla Óptima',
                   labels={'x': 'Mes', 'Error_Absoluto': 'Error Absoluto'},
                   barmode='group',
                   template='plotly_dark')

# Configurar etiquetas de los ejes
fig_error.update_layout(xaxis_title='Mes', yaxis_title='Error Absoluto',
                        xaxis=dict(tickvals=[i for i in range(1, 21)],
                                   ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                             'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

# Mostrar el gráfico
fig_error.show()


<ipython-input-40-dbcaf9544910>:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-40-dbcaf9544910>:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-40-dbcaf9544910>:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass

Ahora que tenemos todos los errores y hemos calculado la "mejor mezcla" la vamos a mostrar.


Evaluamos cuales son las mejores predicciones

In [41]:
# Calcular el error absoluto total para cada predicción
errores_totales = {}
for pred in predicciones:
    # Sumar los errores absolutos mensuales para cada predicción
    errores_totales[pred] = errores_df[f'Error_Absoluto_{pred}'].sum()

# También calcular el error absoluto total de la mezcla óptima
errores_totales['Mezcla_Optima'] = errores_df['Error_Absoluto_Mezcla'].sum()

# Convertir los errores totales en un DataFrame para visualización
errores_totales_df = pd.DataFrame(list(errores_totales.items()), columns=['Tipo_Prediccion', 'Error_Absoluto_Total'])

# Crear el gráfico de barras comparativo del error absoluto total
fig_total_error = px.bar(errores_totales_df,
                         x='Tipo_Prediccion',
                         y='Error_Absoluto_Total',
                         title='Error Absoluto Total por Predicción y Mezcla Óptima',
                         labels={'Tipo_Prediccion': 'Predicción', 'Error_Absoluto_Total': 'Error Absoluto Total'},
                         template='plotly_dark')

# Mostrar el gráfico
fig_total_error.show()


In [42]:
# Extraer la predicción con menor error absoluto (la "mezcla") para cada mes
# Añadir las columnas de predicciones a `errores_df` para filtrar los valores óptimos de predicción

# Combinar errores_df con las predicciones originales para obtener los valores de predicción óptima en cada mes
predicciones_df = ventas_reales_sum.copy()

for pred in predicciones:
    # Filtrar datos de cada predicción y eliminar duplicados
    pred_data = data[((data['YEAR'] == 2023) | ((data['YEAR'] == 2024) & (data['MONTH'] <= 8))) &
                     (data['FORECAST'] == pred) & (data['COUNTRY'] == 'Spain')]
    pred_data_sin_duplicados = eliminar_duplicados_con_mediana(pred_data)
    pred_data_sum = pred_data_sin_duplicados.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

    # Renombrar columna de predicción y añadirla al DataFrame de predicciones
    pred_data_sum = pred_data_sum.rename(columns={'AMOUNT': f'Pred_{pred}'})
    predicciones_df = pd.merge(predicciones_df, pred_data_sum, on=['YEAR', 'MONTH'], how='left')

# Para cada mes, selecciona el valor de la predicción con menor error absoluto
mezcla_valores = errores_df[['YEAR', 'MONTH', 'Prediccion_Optima']].copy()
mezcla_valores['Valor_Prediccion_Mezcla'] = mezcla_valores.apply(lambda row: predicciones_df.loc[
    (predicciones_df['YEAR'] == row['YEAR']) & (predicciones_df['MONTH'] == row['MONTH']),
    f"Pred_{row['Prediccion_Optima']}"
].values[0], axis=1)

# Unir los datos reales con los valores de la mezcla óptima
mezcla_con_reales = pd.merge(ventas_reales_sum, mezcla_valores[['YEAR', 'MONTH', 'Valor_Prediccion_Mezcla']], on=['YEAR', 'MONTH'])

# Crear el gráfico de líneas para comparar las ventas reales con la predicción "mezcla"
fig = px.line(mezcla_con_reales, x=mezcla_con_reales['MONTH'] + (mezcla_con_reales['YEAR'] - 2023) * 12,
              y=['AMOUNT', 'Valor_Prediccion_Mezcla'],
              labels={'value': 'Monto de Ventas', 'variable': 'Serie'},
              title='Comparación de Ventas Reales y Predicción Mezcla Óptima',
              template='plotly_dark')

# Ajustar los nombres de las series en el gráfico
fig.data[0].name = 'Ventas Reales'
fig.data[1].name = 'Predicción Mezcla Óptima'

# Configurar etiquetas de los ejes
fig.update_layout(xaxis_title='Mes', yaxis_title='Monto de Ventas',
                  xaxis=dict(tickvals=[i for i in range(1, 21)],
                             ticktext=['Jan23', 'Feb23', 'Mar23', 'Apr23', 'May23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23',
                                       'Nov23', 'Dec23', 'Jan24', 'Feb24', 'Mar24', 'Apr24', 'May24', 'Jun24', 'Jul24', 'Aug24']))

# Mostrar el gráfico
fig.show()


<ipython-input-40-dbcaf9544910>:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-40-dbcaf9544910>:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-40-dbcaf9544910>:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass

Esta predicción es realmente mejor que las predeterminadas, esto se debe a que hemos elegido la mejor parte de cada predicción en cada caso.

Conclusión sobre el apartado 3:
Inicialmente tan solo había dos predicciones sin valores duplicados lo cuál no no hacía poder ver correctamente todas las predicciones, una vez limpiados,  hemos podido evaluar que predicción era mejor que otra.
Finalmente, hemos hecho la mejor predicción posible mezclando todas las predicciones.